## Coin Tosses
A jar has 1000 coins, of which 999 are fair and 1 is double-headed. Pick a coin at random, and toss it 10 times. Given that you see 10 heads, what is the probability that the next toss of that coin is also a head? Give your answer to 3 significant figures.

Let $D$ be the event that the double-headed coin was picked, and let $F$ be the event that the fair coin was picked

$p(D) = 1/1000$
$p(F) = 999/1000$

Apply Bayes Theorem to calculate the posterior probabilities of each event,

$$P(D|data) = \frac{p(Data|D)\times P(D)}{p(Data|D)\times P(D) + p(Data|F)\times P(F)}$$

$$P(F|data) = \frac{p(Data|F)\times P(F)}{p(Data|F)\times P(F) + p(Data|D)\times P(D)}$$

The previous two formulas show that $P(F|data) = 1-P(D|data)$

Here the data is ten coin tosses resulting in 10 heads.     
Therefore, 
$p(Data|D) = 1^{10} = 1 $  and  $ p(Data|F) = (0.5)^{10} \approx 0.00098$.

Thus, we have all the ingredients to calculate the two posteriors. Next, we need to calculate the probability of getting a head on the 11th toss, which we will do using the law of total probability.

$$p(H) = p(H|D)p(D|data) + p(H|F)(F|data)$$


In [1]:
# Let's write some functions to get the answer.

# prior that the coin is double-headed
pD = 1/1000
# probability of a head given double-headed coin
p_head_D = 1
# prior that the coin is fair
pF = 999/1000
# probability of a head given a fair coin
p_head_F = .5

def lik(p_head, n):
    return p_head**n

def posterior(likA, pA, likB, pB):
    return likA * pA/(likA * pA + likB*pB)

def head_n_plus_one(pHD, pD, pHF, pF, n):
    likD, likF = lik(pHD, n), lik(pHF, n)
    return pHD * posterior(likD, pD, likF, pF) + \
           pHF * posterior(likF, pF, likD, pD)

print('prob of Head on 2nd toss', 
      head_n_plus_one(p_head_D, pD, p_head_F, pF, n=1))
print('prob of Head on 11th toss', 
      head_n_plus_one(p_head_D, pD, p_head_F, pF, n=10))

prob of Head on 2nd toss 0.5009990009990011
prob of Head on 11th toss 0.7530894710825506


In [2]:
# better yet, let's create a class 

class BayesCoinToss():
    """Calculates the posteior probability of a coin given 
    every toss returns a head, where:
    - prior1, the prior probability 
    - prior2, the complement of prior1 (prior2 = 1 - prior1)
    - prob_head1, the probability of obtaining a head given prior1 
    - prob_head2, the probability of obtaining a head given prior2 
    - number_of_flips, the number of coin flips
    
    While prior2 can be calculated from prior1, it is a good idea
    to have both specified for disambiguation of the likelihoods.
    """
    def __init__(self, prior1, prob_head1, prior2,
                 prob_head2, number_of_flips):
        self.prior1 = prior1
        self.prob_head1 = prob_head1
        self.prior2 = prior2
        self.prob_head2 = prob_head2
        self.n_flips = number_of_flips
        # calculate the likelihoods (given each prior)
        self.likelihood1 = prob_head1**number_of_flips
        self.likelihood2 = prob_head2**number_of_flips
    
    def posterior(self):
        """Calculates the posterior probability of a head 
        given the data and the priors
        """
        denominator = self.prior1*self.likelihood1 + self.prior2*self.likelihood2
        self.posterior = self.prior1*self.likelihood1 / denominator
        return self.posterior


In [3]:
PRIOR1 = .001
PROB_HEAD1 = 1
PRIOR2 = .999
PROB_HEAD2 = .5
N = 10

bayes = BayesCoinToss(PRIOR1, PROB_HEAD1, PRIOR2, PROB_HEAD2, N)
posterior = bayes.posterior()

print('p(D), p(H|D), p(data|D):', 
      (bayes.prior1, bayes.prob_head1, bayes.likelihood1))
print('p(F), p(H|F), p(data|F):',
      (bayes.prior2, bayes.prob_head2, bayes.likelihood2))
print('p(D|data) =', posterior)
prob_head_given_data = PROB_HEAD1 * posterior + PROB_HEAD2 * (1 - posterior)
print('Probability of a head on next toss =', prob_head_given_data)

p(D), p(H|D), p(data|D): (0.001, 1, 1)
p(F), p(H|F), p(data|F): (0.999, 0.5, 0.0009765625)
p(D|data) = 0.5061789421651013
Probability of a head on next toss = 0.7530894710825506


## Should I bring an Umbrella to Seattle?

You're about to get on a plane to Seattle. You want to know  if you should bring an umbrella. You call 3 random friends of yours who live there and ask each independently if it's raining. Each of your friends has a 2/3 chance of telling you the truth and a 1/3 chance of messing with you by lying. All 3 friends tell you that "Yes" it is raining. What is the probability that it's actually raining in Seattle?

$p(yes|~\text{rain}) = 2/3$
$p(yes|~\text{no rain}) = 1/3$

$$p(\text{rain}|~ yes, yes, yes)  = \frac{p(yes, yes, yes |~ \text{rain}) \times p(\text{rain})}{p(yes, yes, yes)}\\
= \frac{p(yes, yes, yes |~ rain) \times p(\text{rain})}{p(yes, yes, yes |~ rain) p(\text{rain}) + p(yes, yes, yes |~ \text{no rain}) p(\text{no rain})}$$
 
$$= \frac{(2/3) (2/3) (2/3)\times p(\text{rain})}{(2/3) (2/3) (2/3)\times p(\text{rain}) + (1/3) (1/3) (1/3)\times p(\text{no rain})} \\
=\frac{(8/27)\times p(\text{rain})}{(8/27)\times p(\text{rain}) + (1/27)\times p(\text{no rain})}$$

Suppose that rain/no rain is a 50-50 chance type of a scenario. Thus $p(\text{rain}) = p(\text{no rain}) = 0.5$. Thus,

$$p(\text{rain}|~ yes, yes, yes) = \frac{(8/27)(1/2)}{(8/27)(1/2) + (1/27)(1/2)} = \frac{(8/27)}{(9/27)} = \frac{8}{9} = 0.888\dots$$

In [4]:
# notice we can use BayesCoinToss since we have 
# a binary outcome (rain/no rain) and sequence of 
# three rain outcomes

PRIOR1 = 1/2
PROB_HEAD1 = 2/3
PRIOR2 = 1/2
PROB_HEAD2 = 1/3
N = 3

bayes = BayesCoinToss(PRIOR1, PROB_HEAD1, PRIOR2, PROB_HEAD2, N)
posterior = bayes.posterior()
print("probability of rain given", N, "yes's =", posterior)

probability of rain given 3 yes's = 0.8888888888888888
